In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# # 定義將數值轉換為千位的函數
# def format_thousands(x):
#     return f'{x/1000:,.0f}'

# # 定義將數值轉換為萬位的函數
# def format_ten_thousands(x):
#     return f'{x/10000:,.0f}'

# 定義更新標題的函數
def update_title(y_range):
    if y_range[1] > 1000000:  # 如果範圍值大於 1,000,000，則顯示 (百萬人)
        return "<b>總人口數(百萬人)</b>"
    elif y_range[1] > 1000:   # 如果範圍值大於 1,000，則顯示 (千人)
        return "<b>總人口數(千人)</b>"
    else:                      # 其他情況顯示原始標題
        return "<b>總人口數(人)</b>"

# 讀取 CSV 文件
df = pd.read_csv('file_name.csv')

# 獲取 "group" 欄的唯一值
groups = df['VAR'].unique()

# 迴圈分組繪圖
for group in groups:
    # 篩選特定分組的資料
    filtered_data = df[df['VAR'] == group]

    # 创建图表对象
    fig = go.Figure()

    # 创建线条对象
    traces = []
    colors = ['#4472C4', '#ED7D31', '#70AD47', '#FFC000', '#000000']  # 色號
    for i, column in enumerate(filtered_data.columns[3:8]):  # 從第二欄開始，因為第一欄是年份
        trace = go.Scatter(
            x=filtered_data['period_ad'],
            y=filtered_data[column],
            mode='lines+markers',
            name=column,
            line=dict(color=colors[i], width=2),  # 根據指定的色號為每條線條設置顏色
            marker=dict(symbol='circle', size=8)  # 添加圓點圖徵
        )
        traces.append(trace)

    # 添加第二Y軸資料
    trace_pop = go.Scatter(
        x=filtered_data['period_ad'],
        y=filtered_data['pop_total'],
        mode='lines+markers',
        name='pop_total',
        yaxis='y2',
        line=dict(color='#8B0000', width=2, dash='dash'),  # 指定線的顏色和粗細，這裡使用暗紅色的色號
        marker=dict(symbol='circle', size=8)
    )
    traces.append(trace_pop)

    # 計算第二Y軸的範圍值
    # y_range = [min(filtered_data['pop_total']), max(filtered_data['pop_total'])]
    y_range = [max(filtered_data['pop_total'])]

    # 更新第二Y軸的標題
    y2_title = update_title(y_range)

    # 找到日期數據的最小值和最大值
    min_date = filtered_data['period_ad'].min()
    max_date = filtered_data['period_ad'].max()

    # 創建刻度值和對應的顯示文字
    date_range = pd.date_range(start=min_date, end=max_date, freq='1Y')  # 每2年顯示一次
    tickvals = date_range.tolist()
    ticktext = [date.strftime('%m-%Y') for date in tickvals]

    # 创建布局
    layout = go.Layout(
        width=1024,  # 設定寬度為XXXX像素
        height=576,  # 設定高度為YYY像素
        title=dict(text=f'<b>Graph Title</b>',
                   x=0.5, y=0.925, xanchor='center', font=dict(size=28, color='black')),  # 將標題置中
        xaxis=dict(
            title='<b>X title</b>',
            color='black',
            showline=False,
            showgrid = False,
            linewidth=2,
            mirror=True,
            zeroline=True,
            zerolinecolor='black',
            # gridcolor='black', # 關掉垂直網格線
            titlefont=dict(size=18),
            tickfont=dict(size=14),
            tickangle=90,
            ticktext=ticktext,  # 設定對應的顯示文字
            tickvals=tickvals  # 設定日期刻度值
            # tickmode='auto',
            # tickformat='%Y-%m'
            # tickvals=filtered_data['period_ad'][::1]  # 設定每隔一年顯示一次
        ),
        yaxis=dict(
            title='<b>Y1 title</b>',
            color='black',
            showline=False,
            linewidth=2,  # 調整線條粗細
            # gridwidth=1.5,   # 調整格線寬度
            mirror=True,
            zeroline=False,
            zerolinecolor='black',
            gridcolor='black',
            # tickvals=list(range(0, 100, 10)),
            titlefont=dict(size=18),
            tickfont=dict(size=14)  # 調整字體大小
        ),
        yaxis2=dict(
            # title='<b>Y2 Title</b>',
            title=y2_title,
            color='#8B0000',  # 使用暗紅色的色號
            overlaying='y',
            side='right',
            showline=False,
            showgrid = False,
            linewidth=2,  # 調整線條粗細
            gridwidth=0,   # 調整格線寬度
            mirror=True,
            zeroline=False,
            zerolinecolor='red',
            gridcolor='red',
            titlefont=dict(size=18),
            tickfont=dict(size=14),  # 調整字體大小
            tickvals=None,  # 無需指定 tickvals
            # tickformat=".2f"  # 使用簡短形式顯示，萬位
            # tickvals=list(range(0, 6000, 1000)),
            # ticktext=[format_thousands(val) for val in range(0, 6000, 1000)]
        ),
        plot_bgcolor='white',
        hovermode='x unified',  # 設定浮標線
        legend=dict(x=0.5, y=-0.31, orientation='h', xanchor='center',
                    yanchor='middle', font=dict(size=18)),  # 將图例置于 X 轴标题下方并往下移
    )

    # 将图表对象添加到图表中
    fig.add_traces(traces)
    fig.update_layout(layout)

    # 添加黑色外框
    fig.add_shape(
        type='rect',
        xref='paper', yref='paper',
        x0=0, y0=0, x1=1, y1=1,
        line=dict(color='black', width=1.5),
        fillcolor='rgba(0,0,0,0)',  # 设置透明背景
        layer='below'
    )

    # # 模拟次要格线，每2个单位显示一个次要格线
    # for y_val in range(72, 200, 2):
    #     fig.add_shape(
    #         type='line',
    #         xref='paper', yref='y',
    #         x0=0, y0=y_val, x1=1, y1=y_val,
    #         line=dict(color='lightgrey', width=1),
    #         layer='below'
    #     )

    # 添加浮標線
    fig.update_layout(hovermode='x unified')
    fig.update_xaxes(ticks="outside") # 加x軸刻度，inside在內、outside在外
    #fig.update_yaxes(ticks="inside", col=1) # col=畫subplot時設定第幾張加刻度
    # 输出为 HTML 文件
    pio.write_html(fig, f'Big Title.html')

    # 输出为 JPG 图片文件
    # pio.write_image(fig, f'Big Title.png', format='png') #, width=1360, height=800)

    # 显示图表
    fig.show()

    # 清空图表对象
    fig = None